# Cleaning Kworb Data

In [2]:
# Imports
import pandas as pd

In [39]:
# Load in kworb data
raw_kword_df = pd.read_csv('/Users/joseflemker/Documents/GitHub/3250-data-analytics-template/data/rawstream_dataRaw.csv')

In [40]:
# View data
raw_kword_df.head(5)

,Artist/Song,Streams,Daily
0,The Weeknd - Blinding Lights,"4,560,739,811","1,912,741"
1,Ed Sheeran - Shape of You,"4,096,124,219","1,615,655"
2,Lewis Capaldi - Someone You Loved,"3,671,556,765","1,475,856"
3,Harry Styles - As It Was,"3,631,721,146","1,533,219"
4,Post Malone - Sunflower,"3,595,013,796","1,461,811"


In [41]:
# Check Data types
print(raw_kword_df.dtypes)

Artist/Song    object
Streams        object
Daily          object
dtype: object


In [42]:
# Change Streams and Daily to int
raw_kword_df['Streams'] = raw_kword_df['Streams'].str.replace(',', '')
raw_kword_df['Daily'] = raw_kword_df['Daily'].str.replace(',', '')
raw_kword_df['Streams'] = raw_kword_df['Streams'].astype(float)
raw_kword_df['Daily'] = raw_kword_df['Daily'].astype(float)

In [43]:
# Check for duplicates and null values
duplicates = raw_kword_df.duplicated()
print(f'Duplicate count: {duplicates.sum()}')

Duplicate count: 0


In [44]:
# Check for artist/song duplicates
artistSong_duplicates = raw_kword_df['Artist/Song'].duplicated(keep=False)
print(f'Duplicate artist/song count: {artistSong_duplicates.sum()}')

Duplicate artist/song count: 50


In [45]:
# Examine duplicates
print(raw_kword_df[artistSong_duplicates])

                                     Artist/Song       Streams      Daily
42                      The Weeknd - Die For You  2.512822e+09  1639108.0
80                         Dua Lipa - Levitating  2.190634e+09   855918.0
86                  The Weeknd - Save Your Tears  2.164949e+09  1105282.0
166                          Sia - Cheap Thrills  1.845318e+09   647657.0
222                         Ed Sheeran - Shivers  1.691417e+09  1017947.0
227                 The Weeknd - Save Your Tears  1.687729e+09   874623.0
250   White Noise Baby Sleep - Clean White Noise  1.620154e+09   308720.0
281                    Lil Nas X - Old Town Road  1.559879e+09   199839.0
295              Coldplay - Hymn for the Weekend  1.540262e+09   972993.0
308                             Rema - Calm Down  1.525813e+09   757266.0
414   White Noise Baby Sleep - Clean White Noise  1.361774e+09        NaN
420                         Ed Sheeran - Shivers  1.356315e+09        NaN
734                    Lil Nas X - Old

In [46]:
# Check for null values
print(f'Null artists/songs: {raw_kword_df['Artist/Song'].isna().sum()}')
print(f'Null streams: {raw_kword_df['Streams'].isna().sum()}')
print(f'Null daily streams: {raw_kword_df['Daily'].isna().sum()}')

Null artists/songs: 0
Null streams: 0
Null daily streams: 4


In [47]:
# Examine Null daily streams
raw_kword_df[raw_kword_df['Daily'].isna()]

,Artist/Song,Streams,Daily
414,White Noise Baby Sleep - Clean White Noise,1.361774e+09,NaN
420,Ed Sheeran - Shivers,1.356315e+09,NaN
733,Brenda Lee - Rockin' Around The Christmas Tree,1.040733e+09,NaN
1895,Lil Tjay - F.N,6.247091e+08,NaN


In [48]:
# Check if null values are duplicates
white_noise = raw_kword_df[raw_kword_df['Artist/Song'] == "White Noise Baby Sleep - Clean White Noise"]

shivers = raw_kword_df[raw_kword_df['Artist/Song'] == "Ed Sheeran - Shivers"]

christmas = raw_kword_df[raw_kword_df['Artist/Song'] == "Brenda Lee - Rockin' Around The Christmas Tree"]

lilTjay = raw_kword_df[raw_kword_df['Artist/Song'] == "Lil Tjay - F.N"]
white_noise, shivers, christmas, lilTjay

(                                    Artist/Song       Streams     Daily
 250  White Noise Baby Sleep - Clean White Noise  1.620154e+09  308720.0
 414  White Noise Baby Sleep - Clean White Noise  1.361774e+09       NaN,
               Artist/Song       Streams      Daily
 222  Ed Sheeran - Shivers  1.691417e+09  1017947.0
 420  Ed Sheeran - Shivers  1.356315e+09        NaN,
                                         Artist/Song       Streams  Daily
 733  Brenda Lee - Rockin' Around The Christmas Tree  1.040733e+09    NaN,
          Artist/Song      Streams     Daily
 1304  Lil Tjay - F.N  768520043.0  259796.0
 1895  Lil Tjay - F.N  624709072.0       NaN)

Because the second duplicate is null, if there is one, we will drop this one. We will also give all other duplicates the benifit of the doubt and drop the second version listed and keep the first. This is done because that must be a more unpopular version and we would want to capture the best performance.

In [79]:
# Drop duplicates
kworb_noDupe_df = raw_kword_df.drop_duplicates(keep='first')

duplicates2 = kworb_noDupe_df['Artist/Song'].duplicated()

print(f'New duplicates: {duplicates.sum()}')


New duplicates: 0


For Rocking around the Christmas tree song we are going to impute that value based on the number of total streams

In [80]:
# We need to find the mean daily streams for music with 1,000,000 to 1,100,000 million streams
ranged_streams = kworb_noDupe_df[(kworb_noDupe_df['Streams'] > 1000000000) & (kworb_noDupe_df['Streams'] < 1100000000)]

In [81]:
# Calculate mean for daily
mean_daily_ranged = ranged_streams['Daily'].mean()

In [82]:
# Impute value
kworb_noDupe_df.loc[733, 'Daily'] = mean_daily_ranged.round()

In [83]:
# Examine new value
kworb_noDupe_df.loc[733]

Artist/Song    Brenda Lee - Rockin' Around The Christmas Tree
Streams                                          1040733345.0
Daily                                                621432.0
Name: 733, dtype: object

# Now we split artist and song

In [84]:
# Split artist and song and make new column
kworb_noDupe_df['Artist/Song'] = kworb_noDupe_df['Artist/Song'].str.split(' - ')
kworb_noDupe_df['Song'] = kworb_noDupe_df['Artist/Song']

In [85]:
# Select the artist for the first column and Song for the second column
kworb_noDupe_df['Artist/Song'] = kworb_noDupe_df['Artist/Song'].str[0]
kworb_noDupe_df['Song'] = kworb_noDupe_df['Song'].str[1]

In [86]:
# Examine new Data frame
print(kworb_noDupe_df.head(10))

         Artist/Song       Streams      Daily               Song
0         The Weeknd  4.560740e+09  1912741.0    Blinding Lights
1         Ed Sheeran  4.096124e+09  1615655.0       Shape of You
2      Lewis Capaldi  3.671557e+09  1475856.0  Someone You Loved
3       Harry Styles  3.631721e+09  1533219.0          As It Was
4        Post Malone  3.595014e+09  1461811.0          Sunflower
5         The Weeknd  3.580695e+09  1852199.0            Starboy
6              Drake  3.415975e+09  1335061.0          One Dance
7  The Neighbourhood  3.347043e+09  2882322.0    Sweater Weather
8      The Kid LAROI  3.345711e+09  1582934.0               STAY
9    Imagine Dragons  3.213200e+09  1338922.0           Believer


In [90]:
# Rename and reorder columns
kworb_noDupe_df = kworb_noDupe_df.rename(columns={
    "Artist/Song": "Artist",
    "Streams": "Total Streams",
    "Daily": "Daily Streams"
})
# Reorder
kworb_noDupe_df = kworb_noDupe_df[['Song', 'Artist', 'Total Streams', 'Daily Streams']]

In [91]:
kworb_noDupe_df

,Song,Artist,Total Streams,Daily Streams
0,Blinding Lights,The Weeknd,4.560740e+09,1912741.0
1,Shape of You,Ed Sheeran,4.096124e+09,1615655.0
2,Someone You Loved,Lewis Capaldi,3.671557e+09,1475856.0
3,As It Was,Harry Styles,3.631721e+09,1533219.0
4,Sunflower,Post Malone,3.595014e+09,1461811.0
...,...,...,...,...
2495,Started From the Bottom,Drake,5.203854e+08,113274.0
2496,Shirt,SZA,5.202964e+08,382773.0
2497,Love$ick,Mura Masa,5.201422e+08,143273.0
2498,Forever After All,Luke Combs,5.196419e+08,368956.0
